In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import torchvision.transforms as transforms

import torch
from torch import nn
import glob

import os
import numpy as np
import torch
from PIL import Image
import pandas as pd
import ast
import torch
from path import Path
from torchvision.transforms import transforms
import torchvision
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import cv2
#import torch.utils.data.Dataset 

# to resize bbox along with images
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## preprocessing : get train img_ids,labels and bounding boxes

In [ ]:
train_csv = pd.read_csv("/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
print("Train Data Size : {}".format(train_csv.shape[0]))
train_csv.head()

In [ ]:
# #for debugging
# train_csv = train_csv.dropna()
# train_csv.head()

In [ ]:
# set min x and y to 0 for "no finding"    

for y in ["x_min","y_min"]:
    print(y)
    train_csv[y] = train_csv[y].fillna(0.0)

    
# set max x and y to 1 for "no finding"    

for y in ["x_max","y_max"]:
    print(y)
    train_csv[y] = train_csv[y].fillna(1.0)
    
    
train_csv.head()

rcnn takes 0 for "no finding"
our data has 14 for "no finding"

map org 14 to 0 for rcnn
map org 0 - 13 class ids to 1-14 for rcnn

i.e now 0 means " no finding"  and class ids of "1-14" -> class ids "0-13" in org df

In [ ]:
train_csv["class_id"] = train_csv["class_id"].apply(lambda x : x+1)
train_csv.head()

In [ ]:
#now make id of 15 i,e nofinding to id of 0
train_csv["class_id"] = train_csv["class_id"].replace(15,0)
train_csv.head()

In [ ]:
## see more data

train_csv.iloc[:,0:100]

In [ ]:
#for debugging:

# train_csv["x_diff"] = train_csv["x_max"] - train_csv["x_min"]
# train_csv["y_diff"] = train_csv["y_max"] - train_csv["y_min"]
# print(
# train_csv["x_diff"].min(),

# train_csv["y_diff"].min(),)

# #dropping where diff is less than, = 20
# index_names = train_csv[ train_csv['x_diff'] <= 20 ].index
# print(index_names)
# train_csv.drop(index_names, inplace = True)

# index_names = train_csv[ train_csv['y_diff'] <= 20 ].index
# print(index_names)
# train_csv.drop(index_names, inplace = True)

# train_csv.head()
# # train_csv.loc[1522]
# print(
# train_csv["x_diff"].min(),

# train_csv["y_diff"].min(),)

In [ ]:
train_csv.head()


## dataset class

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
from torch.utils.data import Dataset


class MyDataSet(Dataset):
        
        def __init__(self,data_frame, transforms,image_dir):
            
            super().__init__()
            self.data_frame = data_frame
            self.transforms = transforms
            self.image_dir = Path(image_dir)
            self.image_ids = self.data_frame['image_id'].unique()
            
            
        def __getitem__(self,idx):    # what is this idx?
            
            image_id = self.image_ids[idx]
            image_name = image_id + ".dicom"
            
            # get all images for this image id
            records = self.data_frame[self.data_frame["image_id"]==image_id]
            
            # --------------- preprocess image , method 1 ------------------- #
#             dicom_img = pydicom.read_file(self.image_dir+"/"+image_name)
#             image = dicom_img.pixel_array
            
            # image preprocessing : taken from -> https://www.kaggle.com/chekoduadarsh/pytorch-beginner-code-faster-rcnn
#             if "PhotometricInterpretation" in dicom:
#                 if dicom.PhotometricInterpretation == "MONOCHROME1":
#                     image = np.amax(image) - image

#             intercept = dicom.RescaleIntercept if "RescaleIntercept" in dicom else 0.0
#             slope = dicom.RescaleSlope if "RescaleSlope" in dicom else 1.0

#             if slope != 1:
#                 image = slope * image.astype(np.float64)
#                 image = image.astype(np.int16)

        
#             image += np.int16(intercept)        

#             image = np.stack([image, image, image])
#             image = image.astype('float32')
#             image = image - image.min()
#             image = image / image.max()
#             image = image * 255.0
#             image = image.transpose(1,2,0)



             # --------------- preprocess image , method 2  ------------------- #    
         
#         set keep_ratio=True to have original aspect ratio
#             xray_image = read_xray(self.image_dir+"/"+image_name)
#             image = resize(xray_image, size=512) 
            
            dicom_img = pydicom.read_file(self.image_dir+"/"+image_name)
            image = dicom_img.pixel_array

            image = read_xray(self.image_dir+"/"+image_name)
            
#              wont reszie bbox
            if self.transforms is not None:
                image = self.transforms(image)
       
            
#             bounding boxes
            boxes = records[['x_min','y_min','x_max','y_max']].values
              
#             print("tyoe must be {}".format(type(boxes)))
            boxes = torch.tensor(boxes,dtype  = torch.float64)

            labels = torch.tensor(records["class_id"].values,dtype = torch.int64)
            img_id = torch.tensor([idx])

            
            target={}
            target["img_id"] = img_id
            target["boxes"] = boxes
            target["labels"] = labels


            
            
            return image, target, image_id
        
        def __len__(self):
            return self.image_ids.shape[0]

In [ ]:
train_dir_path = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train"


# wont reszie bbox
mytransforms = transforms.Compose([
    transforms.ToTensor()
])


t_resize = A.Compose([
#         A.Flip(0.5),
#         A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=45, p=0.25),
        A.LongestMaxSize(max_size=800, p=1.0),
#         Dilation(),
        # FasterRCNN will normalize.
#         A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


mydataset = MyDataSet(train_csv,mytransforms,train_dir_path)
# mydataset = VinBigDataset(train_csv, train_dir_path, t_resize)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


# split the dataset in train and test set
indices = torch.randperm(len(mydataset)).tolist()
dataset_train = torch.utils.data.Subset(mydataset, indices[:-1000])
dataset_test = torch.utils.data.Subset(mydataset, indices[-1000:])

mydataloader = torch.utils.data.DataLoader(dataset_train, batch_size=4, collate_fn=collate_fn) # for train
test_dataloader = torch.utils.data.DataLoader(dataset_test, batch_size=1, collate_fn=collate_fn)# for test

In [ ]:
img_data, target_data , imid_data = next(iter(mydataloader))

In [ ]:
print(len(img_data))

In [ ]:
#check images data loader working fine
limit = 0
for t_images, t_targets, t_ids in mydataloader:
    print(t_ids)
    print(type(t_targets[0]["boxes"]))
    for imgs in t_images:
        print(imgs.shape)
    limit+=1
    
    if limit == 2:
        break

## visualize some images

In [ ]:
# for t_images, t_targets, t_ids in mydataloader:

#     for i in range(4):
        
#         img = t_images[i].to('cpu').permute(1,2,0).squeeze()
#         print(img.shape)
#         boxes = t_targets[i]['boxes'].to('cpu')
#         print(img.shape)
#         print(boxes)
#         for box in boxes:
#             print("adding box")
#             img = np.asarray(img)
# #             img = cv2.rectangle(img,(box[0],box[1]),(box[2],box[3]),(255,255,255),2)
# #             img = img.rectangle(box, fill ="# ffff33", outline ="red")

#         plt.imshow(img)
#         plt.show()
        
#     break

In [ ]:
#this test code is designed for batchsize = 4, may require some changes accordinlgy

for t_images, t_targets, t_ids in mydataloader:

    for i in range(3):
        print("**********")
        to_show = t_images[i].to('cpu').permute(1,2,0).numpy() # reshaping tenosr and converting to numpy for disply by plt
        boxes = t_targets[i]['boxes'].to('cpu')
        fig,axs = plt.subplots(1,1)

        for box in boxes:
            print("draw bbox")
            print(box)
#             rect = patches.Rectangle((100, 100), 100, 100, linewidth=1, edgecolor='r', facecolor='none')
                  
            rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none')
            axs.add_patch(rect)

            # Add the patch to the Axes
            axs.imshow(to_show)
    
    break # run for first batch only

## model

In [ ]:
mymodel = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
num_classes = 15

# get number of input features for the classifier
in_features = mymodel.roi_heads.box_predictor.cls_score.in_features

mymodel.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

## training

In [ ]:
params = []

for p in mymodel.parameters():
    if p.requires_grad:
        params.append(p)
        
print(params)

In [ ]:
# optimizer = torch.optim.SGD(params, lr = 0.001)

# epochs = 2

In [ ]:
optimizer = torch.optim.SGD(params, lr=0.0001,
                            momentum=0.6, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

epochs = 2

In [ ]:
mymodel.train()
for epoch in range(epochs):
    
    print("********** EPOCH # {} *************".format(epoch+1))
    b_num = 0
    for t_images, t_targets, t_ids in mydataloader:
        print("----- batch # {} ------".format(b_num+1))
        
        if torch.cuda.is_available():
#             print("transferred")
            t_images = [img.to(device) for img in t_images]
            t_targets = [{k:v.to(device) for k,v in t.items()} for t in t_targets]
            mymodel = mymodel.to(device)
        
#         print(t_targets)
        myloss_dict = mymodel(t_images, t_targets) 
        
        print("loss dict is {}".format(myloss_dict))
        
        all_loss = sum(loss for loss in myloss_dict.values())
        loss_value = all_loss.item()
        
        print("loss : {}".format(loss_value))
        
        
        optimizer.zero_grad()
        all_loss.backward()
        optimizer.step()
        b_num+=1
        
    lr_scheduler.step()

## testing

In [ ]:
mymodel.eval()

In [ ]:
# t=0
# for t_images, t_targets, t_ids in test_dataloader:
#     print("----- test # {} ------".format(t+1))

#     if torch.cuda.is_available():
# #             print("transferred")
#         t_images = [img.to(device) for img in t_images]
#         t_targets = [{k:v.to(device) for k,v in t.items()} for t in t_targets]
#         mymodel = mymodel.to(device)


    
#     predicted = mymodel(t_images) 
# #     print(predicted)

#     label = predicted[0]["labels"][0]
#     bbox = predicted[0]["boxes"][0]
#     score = predicted[0]["scores"][0]
    
#     print("predicted label : {}".format(label))
#     print("predicted bbox : {}".format(bbox))
#     print("prediction score : {}".format(score))
    
    
#     print("true label : {}".format(t_targets[0]["labels"]))
#     print("true bbox : {}".format(t_targets[0]["boxes"]))
    
    
    
    
#     t+=1
#     if t==3:
#         break

# # lr_scheduler.step()

## visualize test results

## submission testing

In [ ]:
class MyTestDataSet(Dataset):
        
        def __init__(self,data_frame, transforms,image_dir):
            
            super().__init__()
            self.data_frame = data_frame
            self.transforms = transforms
            self.image_dir = Path(image_dir)
            self.image_ids = self.data_frame['image_id'].unique()
            
            
        def __getitem__(self,idx):    # what is this idx?
            
            image_id = self.image_ids[idx]
            image_name = image_id + ".dicom"
            
            # get all images for this image id
            records = self.data_frame[self.data_frame["image_id"]==image_id]
            
            image = read_xray(self.image_dir+"/"+image_name)
            
#              wont reszie bbox
            if self.transforms is not None:
                image = self.transforms(image)
       
            
            img_id = torch.tensor([idx])

            
            target={}
            
            image = image.to(device)
            
            
       
            
            return image, image_id
        
        def __len__(self):
            return self.image_ids.shape[0]

In [ ]:
sub_csv = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")
print("Train Data Size : {}".format(sub_csv.shape[0]))
sub_csv.head()

In [ ]:
# test images data loader

test_dir_path = "../input/vinbigdata-chest-xray-abnormalities-detection/test"

sub_dataset = MyTestDataSet(sub_csv,mytransforms,test_dir_path)
sub_dataloader = torch.utils.data.DataLoader(sub_dataset, batch_size=1, collate_fn=collate_fn) # for train

In [ ]:
# chech dataloader working ..
c=0
for img,idx in sub_dataloader:
    print(img[0].shape)
    
    c+=1
    
    if c==3:
        break

In [ ]:
sub_preds = {}

In [ ]:
#testing code
t=0
mymodel.eval()
for t_images, t_ids in sub_dataloader:
    print("-------------------------- test # {} ----------------------".format(t+1))
    pred_str = ""
    if torch.cuda.is_available():
#             print("transferred")
        t_images = [img.to(device) for img in t_images]
        t_targets = [{k:v.to(device) for k,v in t.items()} for t in t_targets]
        mymodel = mymodel.to(device)


    
    predicted = mymodel(t_images) 
#     print(predicted)
#     print(t_ids)

    if len(predicted[0]['labels'])==0:
        label = 14
        pred_str = "14 1 0 0 1 1"
        
    else:
        
        label = predicted[0]["labels"]
        bbox = predicted[0]["boxes"].to('cpu').detach().numpy().astype(np.int32)
        score = predicted[0]["scores"]
    
#     print("predicted label : {}".format(label))
#     print("predicted bbox : {}".format(bbox))
#     print("prediction score : {}".format(score))
    
        thr = 10 if len(label)>=10 else len(label)
        
        for i in range(thr):
#             print("add")
            l = label[i]
            bb = bbox[i]
            sc = score[i]
    
            if l == 0:
                l = 14
                pred_str += "14 1 0 0 1 1 "

            else:
                l -= 1
                pred_str += "{} {} {} {} {} {} ".format(l, sc, bb[0],bb[1],bb[2],bb[3])

    sub_preds[str(t_ids[0])] = pred_str
    print("prediction : {}".format(pred_str))
    
    
    t+=1
#     if t==3:
#         break

In [ ]:
print(sub_preds)

In [ ]:
# set preds in sub csv file
my_sub = pd.DataFrame({"image_id":sub_preds.keys(),"PredictionString":sub_preds.values()})
my_sub.iloc[:,:200]

In [ ]:
my_sub.to_csv("my_sub_all_scs1_no_resize.csv",index=False)